<h1 style=\"text-align: center; font-size: 50px;\">✍️ [MLFlow] MNIST with Keras </h1>
This notebook shows how to do a simple image classification using TensorFlow and MNIST(Modified National Institute of Standards and Technology) database of handwritten digits.


## Notebook Overview
- Imports
- Configurations
- Loading and Preprocessing the Data
- Building Model
- Training the Model
- Making inferences
- Logging Model to MLflow
- Fetching the Latest Model Version from MLflow
- Loading the Model and Running Inference


## Imports

In [ ]:
# ------------------------ System Utilities ------------------------
import warnings                         
import logging
from pathlib import Path              

# ------------------------ Data manipulation libraries ------------------------
import pandas as pd                     
import numpy as np

# ------------------------ Visualization Libraries ------------------------ 
import matplotlib.pyplot as plt

# ------------------------ Deep learning framework ------------------------
import keras
from keras.models import load_model
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping

# ------------------------ MLflow Integration ------------------------
import mlflow
from mlflow import MlflowClient
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec, TensorSpec, ParamSchema, ParamSpec
import mlflow.keras

## Configurations

In [ ]:
# Suppress Python warnings
warnings.filterwarnings("ignore")

In [ ]:
# Create logger
logger = logging.getLogger("MNIST_logger")
logger.setLevel(logging.INFO)

formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s", 
                              datefmt="%Y-%m-%d %H:%M:%S")  

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.propagate = False

In [ ]:
# ------------------------- MLflow Experiment Configuration -------------------------
EXPERIMENT_NAME = 'MNIST with TensorFlow 2'
RUN_NAME = "MNIST_Run"
MODEL_NAME = "MNIST_Model"
MODEL_PATH = "model_keras_mnist"

In [ ]:
logger.info('Notebook execution started.')

## Loading and Preprocessing the Data

The MNIST dataset is divided into two categories: training and testing sets. The load_data() function splits into x_train and y_train, coitaining images and their labels, for trainining the model. The other part, x_test and y_test, hold images and their labels for testing the model.

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
y_cat_test = to_categorical(y_test,10)

In [ ]:
y_cat_train = to_categorical(y_train,10)

In [ ]:
x_train = x_train/255
x_test = x_test/255

Reshape to include channel dimension (in this case, 1 channel)

In [ ]:
x_train = x_train.reshape(60000, 28, 28, 1)

In [ ]:
x_test = x_test.reshape(10000,28,28,1)

## Building the Model

In [ ]:
model = Sequential()

# CONVOLUTIONAL LAYER
model.add(Conv2D(filters=32, kernel_size=(4,4),input_shape=(28, 28, 1), activation='relu',))
# POOLING LAYER
model.add(MaxPool2D(pool_size=(2, 2)))

# FLATTEN IMAGES FROM 28 by 28 to 764 BEFORE FINAL LAYER
model.add(Flatten())

# 128 NEURONS IN DENSE HIDDEN LAYER (YOU CAN CHANGE THIS NUMBER OF NEURONS)
model.add(Dense(128, activation='relu'))

# LAST LAYER IS THE CLASSIFIER, THUS 10 POSSIBLE CLASSES
model.add(Dense(10, activation='softmax'))

model.summary()

In [ ]:
# https://keras.io/metrics/
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']) # you can add in additional metrics https://keras.io/metrics/

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=2)

## Training the Model

In [ ]:
history = model.fit(x_train,y_cat_train,epochs=4,validation_data=(x_test,y_cat_test),callbacks=[early_stop])

## Making inferences

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

## Logging Model to MLflow

In [ ]:
class MNISTModel(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        """
        Load keras model.
        """
        try:
            # Load Iris Flower data
            self.model = keras.models.load_model(context.artifacts["model"])

        except Exception as e:
            logger.error(f"Error loading context: {str(e)}")
            raise

    def predict(self, context, model_input, params):
        """
        Computes the predicted digit.
        """
        try:
            model_input = np.array(model_input) /255.0
            predictions = self.model.predict(model_input)

            return predictions
        
        except Exception as e:
            logger.error(f"Error performing prediction: {str(e)}")
            raise
    
    @classmethod
    def log_model(cls, model_name, keras_model):
        """
        Logs the model to MLflow with appropriate artifacts and schema.
        """
        try:
            mlflow.keras.save_model(keras_model, MODEL_PATH)
            # Define input and output schema
            input_schema = Schema([
                TensorSpec(np.dtype(np.float32),(-1, 28, 28)),
                ])
            output_schema = Schema([
                TensorSpec(np.dtype(np.int64),(-1,)),
            ])
            
            # Define model signature
            signature = ModelSignature(inputs=input_schema, outputs=output_schema)
            
            # Log the model in MLflow
            mlflow.pyfunc.log_model(
                model_name,
                python_model=cls(),
                artifacts={
                    "model": MODEL_PATH,           
                },
                signature=signature,
                pip_requirements=["mlflow", "pandas", "scikit-learn", "numpy"]

            )
        except Exception as e:
            logger.error(f"Error logging model: {str(e)}")
            raise   

In [ ]:
logger.info(f'Starting the experiment: {EXPERIMENT_NAME}')

# Set the MLflow experiment name
mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

# Start an MLflow run
with mlflow.start_run(run_name=RUN_NAME) as run:
    # Print the artifact URI for reference
    logging.info(f"Run's Artifact URI: {run.info.artifact_uri}")
    
    # Log the model to MLflow
    MNISTModel.log_model(model_name=MODEL_NAME)

    # Register the logged model in MLflow Model Registry
    mlflow.register_model(
        model_uri=f"runs:/{run.info.run_id}/{MODEL_NAME}", 
        name=MODEL_NAME
    )

logger.info(f'Registered the model: {MODEL_NAME}')

## Fetching the Latest Model Version from MLflow

In [ ]:
# Initialize the MLflow client
client = MlflowClient()

# Retrieve the latest version of the "Iris_Flower_Model" model (not yet in a specific stage)
model_metadata = client.get_latest_versions(MODEL_NAME, stages=["None"])
latest_model_version = model_metadata[0].version  # Extract the latest model version

# Fetch model information, including its signature
model_info = mlflow.models.get_model_info(f"models:/{MODEL_NAME}/{latest_model_version}")

# Print the latest model version and its signature
print(f"Latest Model Version: {latest_model_version}")
print(f"Model Signature: {model_info.signature}")

## Loading the Model and Running Inference

In [ ]:
model = mlflow.pyfunc.load_model(model_uri=f"models:/{MODEL_NAME}/{latest_model_version}")

predictions = model.predict(x_test[:5])

print("\nPrediction:", predictions)

In [ ]:
logger.info('Notebook execution completed.')

Built with ❤️ using [**Z by HP AI Studio**](https://zdocs.datascience.hp.com/docs/aistudio/overview).